# Exercise 5 - Classification Algorithms (BBNN & SVM)

## a) Back Propogation Neural Network

### Program 1 - Implementing BPNN from scratch

#### Part 1 - Defining class for BPNN

In [1]:
import yaml
import numpy as np

In [2]:
def load_yaml(filename):
    with open(filename, 'r') as f:
        contents = yaml.full_load(f)
    return contents

In [3]:
class BPNN:
    def __init__(self,layer_sizes):
        self.layer_sizes = layer_sizes
        self.n = len(layer_sizes)-1
        
    def error(self,predicted, actual):
        error = actual-predicted
        return error @ error
    
    def activation(self,x):
        return 1/(1+np.exp(-x))
    
    def d_activation(self,x):
         return x * (1 - x)
    
    def pad_ones(X):
        pad_width = [(0,0)]*(X.ndim-1) +[(1,0)]
        return np.pad(X,pad_width=pad_width,constant_values=1)
    
    def inputs(self,i):
        return BPNN.pad_ones(self.outputs[i-1])
    
    def set_random_weights(self,seed=1):
        np.random.seed(seed)
        self.weights = [
            np.random.random((o,i+1))
            for i,o in zip(self.layer_sizes,self.layer_sizes[1:])
        ]
        self.outputs = [None]*(self.n+1)
        self.delta = [None]*self.n

    def forward_propagate(self, inputs):
        assert (self.weights is not None),"weights not given"
        self.outputs[-1] = inputs
        for i in range(self.n):
            self.outputs[i] = self.activation(
                self.weights[i] @ self.inputs(i)
            )
        return self.outputs[-2]

    def backward_propagate_error(self, expected):
        for i in range(self.n-1, -1, -1):
            if i == self.n-1:
                errors = expected - self.outputs[i][1:]
            else:
                errors = self.weights[i+1].T[1:] @ self.delta[i+1]
            self.delta[i] = errors * \
                self.d_activation(self.outputs[i])

    def update_weights(self, l_rate):
        for i in range(self.n):
            self.weights[i] += l_rate * \
                self.delta[i][:, np.newaxis] @ self.inputs(i)[np.newaxis, :]

    def fit(self, X_train,y_train, l_rate, n_epoch):
        classes = np.unique(y_train)
        expected = (y_train.reshape(-1,1) == classes).astype(np.uint8)
        for epoch in range(n_epoch):
            sum_error = 0
            for i, row in enumerate(X_train):
                outputs = self.forward_propagate(row)
                sum_error += self.error(outputs, expected[i])
                self.backward_propagate_error(expected[i])
                self.update_weights(l_rate)
            print(f'>epoch={epoch}, lrate={l_rate:.3},'
                  f' error={sum_error:.4}')

    def predict(self, inputs):
        outputs = self.forward_propagate(inputs).argmax()
        return outputs

    def validate(self,X_test,y_test):
        m = X_test.shape[0]
        for i in range(m):
            actual = self.predict(X_test[i])
            expected = y_test[i].astype(int)
            print(f"Expected={expected}, Got={actual}")

#### Part 2 - Loading and processing dataset

In [4]:
dataset = np.array(load_yaml("./datasets/bpnn_dataset.yaml"))
print("Dataset:\n",dataset,sep="\n")
X = dataset[:,:-1]
y = dataset[:,-1]
split_ratio = 0.5
s = int(split_ratio*X.shape[0])
X_train, X_test = X[:s],X[s:]
y_train, y_test = y[:s],y[s:]

Dataset:

[[ 2.7810836   2.550537    0.        ]
 [ 1.46548937  2.36212508  0.        ]
 [ 3.39656169  4.40029353  0.        ]
 [ 1.38807019  1.85022032  0.        ]
 [ 3.06407232  3.00530597  0.        ]
 [ 7.62753121  2.75926224  1.        ]
 [ 5.33244125  2.08862677  1.        ]
 [ 6.92259672  1.77106367  1.        ]
 [ 8.67541865 -0.24206865  1.        ]
 [ 7.67375647  3.50856301  1.        ]
 [ 2.7810836   2.550537    0.        ]
 [ 1.46548937  2.36212508  0.        ]
 [ 3.39656169  4.40029353  0.        ]
 [ 1.38807019  1.85022032  0.        ]
 [ 3.06407232  3.00530597  0.        ]
 [ 7.62753121  2.75926224  1.        ]
 [ 5.33244125  2.08862677  1.        ]
 [ 6.92259672  1.77106367  1.        ]
 [ 8.67541865 -0.24206865  1.        ]
 [ 7.67375647  3.50856301  1.        ]]


#### Part 3 - Implementing BPNN

In [5]:
b = BPNN([2,2,2])
b.set_random_weights(12)
print("\nInitial Weights:\n")
print(*b.weights,sep="\n")
print("\nTraining:\n")
b.fit(X_train,y_train,.5, 20)
print("\nTrained Weights:\n")
print(*b.weights,sep="\n")
print("\nValidation:\n")
b.validate(X_test,y_test)


Initial Weights:

[[0.15416284 0.7400497  0.26331502]
 [0.53373939 0.01457496 0.91874701]]
[[0.90071485 0.03342143 0.95694934]
 [0.13720932 0.28382835 0.60608318]]

Training:

>epoch=0, lrate=0.5, error=6.816
>epoch=1, lrate=0.5, error=6.529
>epoch=2, lrate=0.5, error=6.317
>epoch=3, lrate=0.5, error=6.032
>epoch=4, lrate=0.5, error=5.494
>epoch=5, lrate=0.5, error=4.991
>epoch=6, lrate=0.5, error=4.585
>epoch=7, lrate=0.5, error=4.153
>epoch=8, lrate=0.5, error=3.688
>epoch=9, lrate=0.5, error=3.211
>epoch=10, lrate=0.5, error=2.745
>epoch=11, lrate=0.5, error=2.302
>epoch=12, lrate=0.5, error=1.893
>epoch=13, lrate=0.5, error=1.54
>epoch=14, lrate=0.5, error=1.25
>epoch=15, lrate=0.5, error=1.02
>epoch=16, lrate=0.5, error=0.84
>epoch=17, lrate=0.5, error=0.7003
>epoch=18, lrate=0.5, error=0.5918
>epoch=19, lrate=0.5, error=0.507

Trained Weights:

[[-0.37268891  1.01813902 -1.21641517]
 [ 0.93666354 -1.57402888  2.08149948]]
[[-0.39942069 -2.33701476  3.51542347]
 [ 0.2811999   1.2

## b) Support Vector Machine

### Program 1 - Implementing SVM from scratch using CVXOPT

#### AIM
To implement SVM from scratch using CVXOPT.

#### Formula
##### SVM statement in dual form
$$
\newcommand{\vect}[1]{\boldsymbol{\mathbf{#1}}}
\begin{aligned}
    & \min_{\vect{\alpha}}  \frac{1}{2}  \vect{\alpha}^T  \left( \vect{y}\vect{y}^T \circ (\Phi(\vect{X})\Phi(\vect{X})^T \right)  \vect{\alpha} - \vect{1}^T \vect{\alpha}\\
s.t.&  - \alpha_i \leq 0 \\
    & \alpha_i \leq C\\
    & y^T \vect{\alpha} = 0  
\end{aligned}
$$

##### Standard form of a quadratic program
$$
\begin{aligned}
    & \min_{\vect{x}}  \frac{1}{2}  \vect{x}^T  \vect{P}  \vect{x} - \vect{q}^T\vect{x}\\
s.t.&  \vect{G}\vect{x} \leq \vect{h} \\
    & A \vect{x} = \vect{b} 
\end{aligned}
$$

##### Formulization
$$
\begin{align*}
\vect{P}&= \vect{y}\vect{y}^T \circ \Phi(\vect{X})\Phi(\vect{X})^T
& \vect{q}&= - \vect{1}_n\\
\vect{G}&= \begin{bmatrix}
    \vect{-I}_n\\
    \vect{I}_n
\end{bmatrix} 
& \vect{h}&=\begin{bmatrix}
    C \cdot \vect{1}_n^T\\
    \vect{0}_n^T
\end{bmatrix}\\
\vect{A} &= [\vect{y}] 
&\vect{b} &= [0]\\
\end{align*}
$$

##### Decision Rule
$$
\vect{\hat y} = \mathrm{sign}\left( (\vect{\alpha}^T \circ \vect{y}^T) \Phi(\vect{X})\Phi(\vect{X})^T  + b \right) 
$$

#### Part 1 - Defining class for SVM

In [6]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from cvxopt import solvers
from cvxopt import matrix

from scipy.spatial.distance import cdist

In [7]:
class SVM:
    def __init__(self,C,kernel):
        self.C = C
        self.kernel = kernel

    def fit(self,X_train,y_train):
        self.scaler = StandardScaler()
        self.X = self.scaler.fit_transform(X_train)
        self.y = y_train.reshape(-1,1)
        
        n=self.X.shape[0]
        I_n = np.eye(n)
        P=(self.y@self.y.T)*self.kernel(self.X,self.X)
        q=np.full(n,-1)
        G=np.vstack((I_n,-1*I_n))
        h=np.hstack((np.full(n,self.C),np.zeros(n)))
        A=y_train.reshape(1,-1)
        b=np.zeros(1)

        P,q,G,h,A,b = map(lambda x : matrix(x,tc="d"),(P,q,G,h,A,b))

        solution = solvers.qp(P, q, G, h, A, b)
        self.a = np.asarray(solution['x']).squeeze()
        
        support_indices = np.logical_and(self.a>=1e-10, self.a<self.C)
        X_S = self.X[support_indices]
        self.b = np.mean(self.y - self.a*self.y.T @ self.kernel(self.X, X_S))

    def predict(self,X_test):
        X_test=self.scaler.transform(X_test)
        return np.sign(self.a*self.y.T @ self.kernel(self.X, X_test) + self.b)


#### Part 2 - Defining Radial Basis Function(RBF) Kernel

In [8]:
def rbf_kernel(X1,X2,sigma):
    return np.exp(-cdist(X1, X2, 'sqeuclidean') / (2*sigma**2))

#### Part 3 - Loading and Processing Dataset

In [9]:
# Processed titanic dataset from Exercise 2
titanic_df = pd.read_csv('datasets/titanic_processed.csv')
X = titanic_df.drop('Survived',axis = 1).values
y = titanic_df['Survived'].values
y[y==0] = -1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#### Part 4 - Implementing SVM

In [10]:
from functools import partial 
C = .35
sigma = .5
kernel = partial(rbf_kernel,sigma=sigma)

svm_classifier = SVM(C,kernel)
svm_classifier.fit(X_train,y_train)
y_pred = svm_classifier.predict(X_test)
print(f'Accuracy of the classifer: {(y_test == y_pred).mean()*100:.2f}%')


     pcost       dcost       gap    pres   dres
 0: -2.0672e+02 -6.0489e+02  6e+03  8e+00  1e-15
 1: -1.0821e+02 -5.1411e+02  6e+02  3e-01  1e-15
 2: -1.0989e+02 -1.6099e+02  5e+01  9e-03  2e-15
 3: -1.2250e+02 -1.3135e+02  9e+00  1e-03  1e-15
 4: -1.2441e+02 -1.2830e+02  4e+00  4e-04  1e-15
 5: -1.2525e+02 -1.2680e+02  2e+00  1e-04  1e-15
 6: -1.2558e+02 -1.2625e+02  7e-01  7e-06  1e-15
 7: -1.2572e+02 -1.2596e+02  2e-01  9e-07  1e-15
 8: -1.2577e+02 -1.2588e+02  1e-01  4e-07  1e-15
 9: -1.2578e+02 -1.2586e+02  8e-02  2e-07  9e-16
10: -1.2580e+02 -1.2582e+02  2e-02  2e-08  1e-15
11: -1.2581e+02 -1.2581e+02  6e-03  6e-09  1e-15
12: -1.2581e+02 -1.2581e+02  3e-03  1e-09  1e-15
13: -1.2581e+02 -1.2581e+02  9e-04  3e-10  9e-16
14: -1.2581e+02 -1.2581e+02  3e-04  9e-11  9e-16
15: -1.2581e+02 -1.2581e+02  2e-05  3e-15  1e-15
Optimal solution found.
Accuracy of the classifer: 73.03%
